<a href="https://colab.research.google.com/github/JD-man/GAN_Study/blob/main/8.SGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SGAN

### 준지도학습
- 훈련 데이터셋의 일부에만 클래스 레이블을 가지고 있다.
- 데이터에 감춰진 내부 구조를 사용해 일부 레이블된 데이터 포인트를 일반화한다.
- 효율적인 방식으로 이전에 본 적 없는 새로운 샘플을 분류한다.
- 레이블된 데이터와 레이블이 없는 데이터가 동일한 분포에서 수집되어야 한다.
- 생성 모델을 사용하여 훈련에 사용할 수 있는 추가 정보를 제공하면 준지도 학습 모델의 정확도 향상에 도움을 준다.

### SGAN이란?
- semi-supervised generative adversarial network
- 판별자가 다중 분류를 수행하는 생성적 적대 신경망
- 진짜와 가짜 두개의 클래스만 구별하는 것이 아니라 N + 1개의 클래스를 구별하도록 학습한다.
- N : 훈련 샘플에 있는 클래스 개수 , 1 : 생성자가 만든 가짜 샘플
- 판별자의 출력이 클래스에 대한 확률 벡터로 표현되고, 진짜인지 가짜인지 확률이 하나 더 있다.

### SGAN의 구조
- 생성자는 오리지널 GAN과 동일. 랜덤한 벡터를 받아 훈련 세트와 구분이 안되는 가짜 샘플을 만든다.
- 판별자는 오리지널 GAN과 다름. 둘이 아니라 세 종류의 입력을 받는다.
- 생성자가 만든 가짜샘플 x<sup>*</sup>, 훈련 데이터셋에서 레이블이 없는 진짜 샘플 x, 훈련 데이터셋에서 레이블이 있는 진짜 샘플 (x,y). 이 세 종류의 입력을 받는다.
- 판별자의 목표는 입력 샘플이 진짜일 경우 해당하는 클래스로 분류하고 아닐 경우 가짜로 처리하는 것이다.

### 훈련과정
- 판별자 D(x)와 D(x<sup>*</sup>에 대한 손실과 지도 학습 훈련 샘플 D((x,y))에 대한 손실도 계산해야한다.
- 원래 논문의 용어를 사용하면 이런 이중목표는 지도 손실과 비지도 손실이라는 두 종류의 손실에 해당한다.

### 훈련목표
- 일반적인 GAN은 생성 모델이다. 훈련이 끝나면 판별자는 버리고 훈련이 끝난 생성자만 사용하여 실제와 같은 합성 데이터를 만든다.
- SGAN에서는 판별자가 주요 관심 대상이다.
- 훈련 목표는 적은 양의 레이블이 있는 데이터만 사용하여 높은 분류 정확도를 가지는 판별자를 만드는 것이다.
- SGAN에서 생성자는 판별자의 훈련을 도우는 역할이며, 훈련이 끝난 후 생성자를 버리고 훈련된 판별자를 분류기로 사용한다.

# SGAN의 구현
MNIST 데이터셋에서 100개의 훈련 샘플만 사용해 손글씨 숫자를 분류하는 SGAN을 만든다.

### 모델의 구조
- 진짜일 경우 데이터를 분류하는 다중 분류 문제를 풀기 위해 판별자는 소프트맥스 함수를 사용한다.
- 분류 오차를 계산하기 위해 출력 확률과 원핫 인코딩된 타깃 레이블 사이의 교차 엔트로피 손실을 사용한다.
- 진짜인지 가짜인지를 나타내는 확률을 나타내기 위해서는 시그모이드 함수를 사용한다. 손실 함수는 이진 교차 엔트로피를 사용한다.

### 구현
- SGAN 구현의 대부분은 DCGAN에서 따왔다.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras import backend as K
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import (Activation, BatchNormalization, Concatenate, Dense, Dropout, Flatten, Input, Lambda, Reshape)
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [ ]:
# 입력 이미지 크기, 잡음 벡터 z의 크기, 진짜 클래스 개수 지정

img_rows = 28
img_cols = 28
channels = 1

img_shape = (img_rows, img_cols, channels)
z_dim = 100
num_classes = 10

In [ ]:
# 데이터셋 가져오기. 50000개 중 일부만 훈련에 사용하고 나머지는 레이블이 없는 것처럼 다룬다.

class Dataset:
  def __init__(self, num_labeled):
    
    # 훈련에 사용할 레이블된 샘플 개수
    self.num_labeled = num_labeled

    # MNIST 데이터
    (self.x_train, self.y_train), (self.x_test, self.y_test) = mnist.load_data()
  
    def preprocess_imgs(x):
    
      # 픽셀값을 -1~1 사이로 변환하고, 이미지 차원을 너비 X 높이 X 채널로 확장
      x = (x.astype(np.float32) - 127.5) / 127.5
      x = np.expand_dims(x, axis=3)
      return x

    def preprocess_labels(y):
      return y.reshape(-1,1)

    self.x_train = preprocess_imgs(self.x_train)
    self.y_train = preprocess_labels(self.y_train)

    self.x_test = preprocess_imgs(self.x_test)
    self.y_test = preprocess_labels(self.y_test)

  # 레이블된 이미지와 레이블을 랜덤 배치 만들기
  def batch_labeled(self, batch_size):
    idx = np.random.randint(0, self.num_labeled, batch_size)
    imgs = self.x_train[idx]
    labels = self.y_train[idx]
    return imgs, labels

  # 레이블 없이 이미지를 랜덤 배치 만들기
  def batch_unlabeled(self, batch_size):
    idx = np.random.randint(self.num_labeled, self.x_train.shape[0], batch_size)
    imgs = self.x_train[idx]
    return imgs
  
  def training_set(self):
    x_train = self.x_train[range(self.num_labeled)]
    y_train = self.y_train[range(self.num_labeled)]
    return x_train, y_train

  def test_set(self):
    return self.x_test, self.y_test


In [ ]:
# 이 튜토리얼에서는 레이블된 MNIST 이미지 100개만 훈련한다

num_labeled = 100
dataset = Dataset(num_labeled)

In [ ]:
#생성자 만들기 함수. DCGAN에서 만든 것과 동일

def build_generator(z_dim):
  model = Sequential()
  model.add(Dense(256 * 7 * 7, input_dim = z_dim))
  model.add(Reshape((7,7,256)))
  model.add(Conv2DTranspose(128, kernel_size=3, strides=2, padding='same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.01))
  model.add(Conv2DTranspose(64, kernel_size=3, strides=1, padding='same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.01))
  model.add(Conv2DTranspose(1, kernel_size=3, strides=2, padding='same'))
  model.add(Activation('tanh'))
  return model

In [ ]:
# 판별자 만들기 함수. 판별자는 두가지 목표를 갖는다.

# 1. 진짜와 가짜 샘플 구별. 시그모이드 함수를 사용해 확률 출력
# 2. 진짜 샘플일 경우 레이블을 분류. 소프트맥스 함수를 사용해 클래스마다 하나씩 확률을 출력


def build_discriminator_net(img_shape):
  model = Sequential()
  model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding='same'))
  model.add(LeakyReLU(alpha=0.01))
  model.add(Conv2D(64, kernel_size=3, strides=2, input_shape=img_shape, padding='same'))
  model.add(LeakyReLU(alpha=0.01))
  model.add(Conv2D(128, kernel_size=3, strides=2, input_shape=img_shape, padding='same'))
  model.add(LeakyReLU(alpha=0.01))
  model.add(Dropout(0.5))
  model.add(Flatten())
  model.add(Dense(num_classes))
  return model
# 먼저 판별자의 출력을 계산하는 신경망을 정의했다
# 이 10개의 출력으로 하나는 지도학습 다중분류. 다른 하나는 비지도학습 이진분류를 수행한다.

In [ ]:
# 지도학습 판별자 만들기

def build_discriminator_supervised(discriminator_net):
  model = Sequential()
  model.add(discriminator_net)
  # 진짜 샘플일 경우 클래스에 대한 예측 확률을 출력하는 소프트맥스 활성화 함수
  model.add(Activation('softmax'))
  return model

In [ ]:
# 비지도 학습 판별자

def build_discriminator_unsupervised(discriminator_net):
  model = Sequential()
  model.add(discriminator_net)
  
  def predict(x):
    # 진짜 클래스에 대한 확률 분포를 진짜 대 가짜 이진 확률로 변환한다
    # prediction에 사용한 함수는 케라스에서 사용하는 sigmoid는 아니지만 S자 곡선의 함수이다.
    # 10개 뉴런의 출력에 모두 지수함수를 적용하여 1을 더해 분모에 넣는다.
    # 10개 뉴런 출력이 큰 음수일때는 분수는 1에 가까워져 predict는 0에 가까운값을 가진다.
    prediction = 1.0 - (1.0 / (K.sum(K.exp(x), axis=-1, keepdims=True) + 1.0))
    return prediction
  
  
  model.add(Lambda(predict))
  return model

In [ ]:
# GAN 모델 구성

def build_gan(generator, discriminator):
  model = Sequential()
  model.add(generator)
  model.add(discriminator)
  return model

discriminator_net = build_discriminator_net(img_shape)

discriminator_supervised = build_discriminator_supervised(discriminator_net)
discriminator_supervised.compile(loss = 'categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(learning_rate=0.0003))

discriminator_unsupervised = build_discriminator_unsupervised(discriminator_net)
discriminator_unsupervised.compile(loss='binary_crossentropy', optimizer=Adam())

generator = build_generator(z_dim)

# 진짜 가짜 판별은 생성자 훈련시 동결해야한다
discriminator_unsupervised.trainable = False

# GAN은 생성자와 비지도학습 판별자와 연결한다
gan = build_gan(generator, discriminator_unsupervised)
gan.compile(loss = 'binary_crossentropy', optimizer = Adam())

# SGAN 훈련 알고리즘

1. (지도학습) 판별자를 훈련한다.  
  a. 레이블된 진짜 샘플 (x,y)의 랜덤 미니배치를 얻는다.  
  b. 주어진 미니배치에 대한 D((x,y))를 계산하고 다중 분류 손실을 역전파하여 판별자 파라미터를 업데이트하고 손실을 최소화한다.  
2. (비지도 학습) 판별자를 훈련한다.  
  a. 레이블이 없는 진짜 샘플 x의 랜덤 미니배치를 얻는다.  
  b. 주어진 미니배치에 대한 D(x)를 계산하고 이진 분류 손실을 역전파하여 판별자 파라미터를 업데이트하고 손실을 최소화 한다.  
  c. 랜덤한 벡터 z의 미니배치를 얻어 가짜 샘플의 미니배치 x<sup>*</sup>를 생성한다.  
  d. 주어진 미니배치에 대한 D(x<sup>*</sup>)을 계산하고 이진 분류 손실을 역전파하여 판별자 파라미터를 업데이트하고 손실을 최소화 한다.
3. 생성자를 훈련한다  
  a. 랜덤한 벡터 z의 미니배치를 얻어 가짜 샘플의 미니배치 x<sup>*</sup>를 생성한다.  
  b. 주어진 미니배치에 대한 D(x<sup>*</sup>)을 계산하고 이진 분류 손실을 역전파하여 생성자 파라미터를 업데이트하고 손실을 최대화 한다.

In [ ]:
# SGAN 훈련 알고리즘 구현

supervised_losses = []
iteration_checkpoints = []

def train(iterations, batch_size, sample_interval):
  real = np.ones((batch_size, 1))
  fake = np.zeros((batch_size, 1))

  for iteration in range(iterations):
    imgs, labels = dataset.batch_labeled(batch_size)
    labels = to_categorical(labels, num_classes)
    imgs_unlabeled = dataset.batch_unlabeled(batch_size)
    
    z= np.random.normal(0, 1, (batch_size, z_dim))
    gen_imgs = generator.predict(z)

    d_loss_supervised, accuracy = discriminator_supervised.train_on_batch(imgs, labels)

    d_loss_real = discriminator_unsupervised.train_on_batch(imgs_unlabeled, real)
    d_loss_fake = discriminator_unsupervised.train_on_batch(gen_imgs, fake)
    d_loss_unsupervised = 0.5 * np.add(d_loss_real, d_loss_fake)

    z = np.random.normal(0, 1, (batch_size, z_dim))
    gen_imgs = generator.predict(z)

    g_loss = gan.train_on_batch(z, real)

    if(iteration + 1) % sample_interval == 0:
      supervised_losses.append(d_loss_supervised)
      iteration_checkpoints.append(iteration + 1)

      print("%d [D_sup_loss : %.4f, acc : %.2f%%] [D_unsup_loss : %.4f] [G loss : %f]"
             % (iteration + 1, d_loss_supervised, 100 * accuracy, d_loss_unsupervised, g_loss))

In [ ]:
iterations = 8000
batch_size = 32
sample_interval = 800

train(iterations, batch_size, sample_interval)

800 [D_sup_loss : 0.4019, acc : 93.75%] [D_unsup_loss : 0.3690] [G loss : 1.883652]
1600 [D_sup_loss : 0.1376, acc : 96.88%] [D_unsup_loss : 0.3389] [G loss : 1.815471]
2400 [D_sup_loss : 0.0823, acc : 96.88%] [D_unsup_loss : 0.6284] [G loss : 1.981820]
3200 [D_sup_loss : 0.0250, acc : 100.00%] [D_unsup_loss : 0.4038] [G loss : 1.535358]
4000 [D_sup_loss : 0.0175, acc : 100.00%] [D_unsup_loss : 0.4935] [G loss : 1.808986]
4800 [D_sup_loss : 0.0031, acc : 100.00%] [D_unsup_loss : 0.3059] [G loss : 1.655713]
5600 [D_sup_loss : 0.0108, acc : 100.00%] [D_unsup_loss : 0.5056] [G loss : 1.928070]
6400 [D_sup_loss : 0.0033, acc : 100.00%] [D_unsup_loss : 0.2981] [G loss : 1.619153]
7200 [D_sup_loss : 0.0033, acc : 100.00%] [D_unsup_loss : 0.3714] [G loss : 1.881701]
8000 [D_sup_loss : 0.0005, acc : 100.00%] [D_unsup_loss : 0.2215] [G loss : 2.329058]


In [ ]:
# 정확도 체크

# 테스트세트
x, y = dataset.test_set()
y = to_categorical(y, num_classes)

_, accuracy = discriminator_supervised.evaluate(x,y)
print("정확도 : %.2f%%" % (100 * accuracy))

313/313 [==============================] - 1s 2ms/step - loss: 0.2219 - accuracy: 0.9262
정확도 : 92.62%


In [ ]:
# 완전 지도학습으로 훈련한 모델
# 비교를 위해 SGAN에 사용한 지도학습 모델을 사용

mnist_classifier = build_discriminator_supervised(build_discriminator_net(img_shape))
mnist_classifier.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer=Adam())

In [ ]:
# 비교를 위해 SGAN에 사용한 훈련데이터를 사용

imgs, labels = dataset.training_set()
labels = to_categorical(labels, num_classes)

mnist_classifier.fit(imgs, labels, 32, 30, 1)

Epoch 1/30
4/4 [==============================] - 0s 18ms/step - loss: 2.2832 - accuracy: 0.1461
Epoch 2/30
4/4 [==============================] - 0s 4ms/step - loss: 2.1837 - accuracy: 0.2622
Epoch 3/30
4/4 [==============================] - 0s 4ms/step - loss: 2.1059 - accuracy: 0.2879
Epoch 4/30
4/4 [==============================] - 0s 4ms/step - loss: 1.9956 - accuracy: 0.3283
Epoch 5/30
4/4 [==============================] - 0s 4ms/step - loss: 1.8879 - accuracy: 0.3482
Epoch 6/30
4/4 [==============================] - 0s 4ms/step - loss: 1.7181 - accuracy: 0.4910
Epoch 7/30
4/4 [==============================] - 0s 4ms/step - loss: 1.4331 - accuracy: 0.6612
Epoch 8/30
4/4 [==============================] - 0s 4ms/step - loss: 1.1796 - accuracy: 0.7152
Epoch 9/30
4/4 [==============================] - 0s 4ms/step - loss: 0.8461 - accuracy: 0.8242
Epoch 10/30
4/4 [==============================] - 0s 4ms/step - loss: 0.8433 - accuracy: 0.7630
Epoch 11/30
4/4 [=====================

In [ ]:
# 완전지도 분류기의 테스트 세트에 대한 성능확인

_, accuracy = mnist_classifier.evaluate(x,y)
print("정확도 : %.2f%%" % (100 * accuracy))

313/313 [==============================] - 1s 2ms/step - loss: 1.1450 - accuracy: 0.7175
정확도 : 71.75%


In [ ]:
# 두 모델 모두 훈련데이터에 대해서는 100% 정확도를 가진다.

# SGAN으로 훈련한 분류의 정확도 : 92.62%
# 완전지도로 훈련한 분류의 정확도 : 71.75%

# 같은 레이블된 데이터 100개를 사용했을때 SGAN으로 훈련한 분류기의 성능이 완전지도로 훈련한 분류기의 정확도보다 20% 높다.
# 적은 수의 훈련샘플에서 SGAN으로 훈련한 분류기 성능이 완전 지도 학습 분류기보다 뛰어나다는 것을 보여준다.
# 이는 분류 작업의 대량의 레이블된 데이터셋에 대한 의존성을 줄일 수 있다.